In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd
import skforecast

print(skforecast.__version__)

/home/joaquin/Documents/GitHub/skforecast
0.19.1


In [2]:
import numpy as np
import pandas as pd
import skforecast
from sklearn.datasets import make_regression
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import time

In [3]:
# Data
X, y = make_regression(n_samples=1000, n_features=20, noise=0.1, random_state=42)
X_predict = X[:1].squeeze()
X_predict

array([ 0.22584183,  1.55137772, -0.10734682,  0.85969505, -0.94296337,
       -1.09662504, -1.19716659, -1.73376709, -0.95004188,  1.27405964,
        1.6729897 ,  1.47737256,  0.33512613, -0.6211348 , -1.52510564,
        1.01109701,  1.47818394, -1.35154745,  0.3646018 ,  0.13316649])

In [4]:
results = []
n_iterations = 1000

# Train Ridge model
model_ridge = Ridge(alpha=1.0)
model_ridge.fit(X, y)

# Benchmark sklearn predict
start = time.perf_counter()
for _ in range(n_iterations):
    pred = model_ridge.predict(X_predict.reshape(1, -1))
end = time.perf_counter()
sklearn_time = end - start

# Benchmark numpy dot product
start = time.perf_counter()
for _ in range(n_iterations):
    pred = np.dot(X_predict, model_ridge.coef_) + model_ridge.intercept_
end = time.perf_counter()
numpy_time = end - start

speedup = sklearn_time / numpy_time
results.append({
    'batch_size': 1,
    'sklearn_time': sklearn_time,
    'numpy_time': numpy_time,
    'speedup': speedup
})

# Check that predictions are the same
pred_sklearn = model_ridge.predict(X_predict.reshape(1, -1))
pred_numpy = np.dot(X_predict, model_ridge.coef_) + model_ridge.intercept_
assert np.allclose(pred_sklearn, pred_numpy), "Predictions do not match!"

results_df = pd.DataFrame(results)
print(f"\nPerformance Comparison ({n_iterations} iterations):")
print("="*60)
print(results_df.to_string(index=False))


Performance Comparison (1000 iterations):
 batch_size  sklearn_time  numpy_time   speedup
          1      0.057235     0.00092 62.215184


In [5]:
import warnings

X_predict = X[:1]
n_iterations = 1000

model_lightgbm = LGBMRegressor(n_estimators=100, random_state=42, verbose=-1)
model_lightgbm.fit(X, y)

# Standard predict
start = time.perf_counter()
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for _ in range(n_iterations):
        pred = model_lightgbm.predict(X_predict)
end = time.perf_counter()
standard_time = end - start

# Using the booster directly (lower level API)
start = time.perf_counter()
booster = model_lightgbm.booster_
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for _ in range(n_iterations):
        pred = booster.predict(X_predict)
end = time.perf_counter()
booster_time = end - start

# Check that predictions are the same
pred_standard = model_lightgbm.predict(X_predict)
pred_booster = booster.predict(X_predict)
assert np.allclose(pred_standard, pred_booster), "Predictions do not match!"

print(f"\nLGBMRegressor Performance ({n_iterations} iterations):")
print("="*60)
print(f"Standard predict:           {standard_time:.6f} s ({standard_time/n_iterations*1e6:.2f} µs per pred)")
print(f"Booster direct predict:     {booster_time:.6f} s ({booster_time/n_iterations*1e6:.2f} µs per pred)")
print(f"Speedup (booster vs standard):   {standard_time/booster_time:.2f}x")


LGBMRegressor Performance (1000 iterations):
Standard predict:           0.557721 s (557.72 µs per pred)
Booster direct predict:     0.095860 s (95.86 µs per pred)
Speedup (booster vs standard):   5.82x


/home/joaquin/miniconda3/envs/skforecast_20_py13/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [6]:
X_predict = X[:1]
n_iterations = 1000

model_xgboost = XGBRegressor(n_estimators=100, random_state=42, verbosity=0)
model_xgboost.fit(X, y)

# Standard predict
start = time.perf_counter()
for _ in range(n_iterations):
    pred = model_xgboost.predict(X_predict)
end = time.perf_counter()
standard_time = end - start


# Standard predict
start = time.perf_counter()
for _ in range(n_iterations):
    pred = model_xgboost.predict(X_predict,  validate_features=False)
end = time.perf_counter()
standard_no_validation_time = end - start

# Using the booster directly (lower level API)
start = time.perf_counter()
booster = model_xgboost.get_booster()
for _ in range(n_iterations):
    pred = booster.inplace_predict(X_predict)
end = time.perf_counter()
booster_time = end - start

print(f"\nLGBMRegressor Performance ({n_iterations} iterations):")
print("="*60)
print(f"Standard predict:                 {standard_time:.6f} s ({standard_time/n_iterations*1e6:.2f} µs per pred)")
print(f"Standard predict (no validation): {standard_no_validation_time:.6f} s ({standard_no_validation_time/n_iterations*1e6:.2f} µs per pred)")
print(f"Booster direct predict:           {booster_time:.6f} s ({booster_time/n_iterations*1e6:.2f} µs per pred)")
print(f"Speedup (booster vs standard):    {standard_time/booster_time:.2f}x")


LGBMRegressor Performance (1000 iterations):
Standard predict:                 1.006903 s (1006.90 µs per pred)
Standard predict (no validation): 0.536726 s (536.73 µs per pred)
Booster direct predict:           0.424210 s (424.21 µs per pred)
Speedup (booster vs standard):    2.37x


In [7]:
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from skforecast.datasets import load_demo_dataset
from skforecast.preprocessing import RollingFeatures
from skforecast.recursive import ForecasterRecursive

data = load_demo_dataset()
data.head(5)

datetime
1991-07-01    0.429795
1991-08-01    0.400906
1991-09-01    0.432159
1991-10-01    0.492543
1991-11-01    0.502369
Freq: MS, Name: y, dtype: float64

In [8]:
forecaster = ForecasterRecursive(
                 estimator       = LGBMRegressor(random_state=123, verbose=-1),
                 lags            = 48,
             )

forecaster.fit(y=data)

In [9]:
%%timeit
forecaster.predict(steps=500)

56.8 ms ± 6.65 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
forecaster = ForecasterRecursive(
                 estimator       = Ridge(),
                 lags            = 48,
             )

forecaster.fit(y=data)

In [11]:
%%timeit
forecaster.predict(steps=500)

5.55 ms ± 395 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
forecaster = ForecasterRecursive(
                 estimator       = XGBRegressor(random_state=123, verbosity=0),
                 lags            = 48,
             )

forecaster.fit(y=data)

In [13]:
%%timeit
forecaster.predict(steps=500)

211 ms ± 49.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
